In [1]:
from differences import ATTgt
from matplotlib import pyplot as plt
from analysis_utilities import aggregate_by_time_and_plot, aggregate_by_event_time_and_plot, \
    select_controls, test_balance, add_missing_indicators, prepare_df, generate_variable_names

plt.rcParams["figure.dpi"] = 300
plt.rcParams['savefig.dpi'] = 300
import pandas as pd

# Store paths.
INPUT_DATA = "/Users/arjunshanmugam/Documents/GitHub/seniorthesis/data/03_cleaned/crime_analysis.csv"
OUTPUT_FIGURES = "/Users/arjunshanmugam/Documents/GitHub/seniorthesis/output/crime_200m/figures"
OUTPUT_TABLES = "/Users/arjunshanmugam/Documents/GitHub/seniorthesis/output/crime_200m/tables"


# Read restricted sample panel dataset into memory.
df = pd.read_csv(INPUT_DATA)

analysis='group_2_crimes_50m'

In [2]:
import numpy as np

value_vars_crime, month_to_int_dictionary, int_to_month_dictionary = generate_variable_names(analysis=analysis)

# Uncomment below line for "any crime" analysis
# df[value_vars_crime] = np.where(df[value_vars_crime] > 0, 1, 0)

In [3]:
covariates_exploration_df = select_controls(df=df, analysis=analysis, output_directory=OUTPUT_TABLES)
covariates_exploration_df

Dependent Variable  \
                                                                                            Drug-Related Crime Incidents Within 50m, Oct. 2022   
                                                 \emph{Independent Variable}                                                                     
Panel A: Pre-treatment Outcomes                  twenty_seventeen_group_2_crimes_50m                                                       NaN   
                                                 pre_treatment_change_in_group_2_crimes_50m                                                NaN   
Panel B: Census Tract Characteristics            frac_coll_plus2010                                                                        NaN   
                                                 job_density_2013                                                                          NaN   
                                                 med_hhinc2016                                                                             NaN   
                                                 poor_share2010                                                                            NaN   
                                                 popdensity2010                                                                            NaN   
                                                 rent_twobed2015                                                                           NaN   
                                                 share_white2010                                                                           NaN   
                                                 traveltime15_2010                                                                         NaN   
Panel C: Case Initiation                         for_cause                                                                                 NaN   
                                                 no_cause                                                                                  NaN   
                                                 non_payment                                                                               NaN   
Panel D: Defendant and Plaintiff Characteristics hasAttyD                                                                                  NaN   
                                                 hasAttyP                                                                                  NaN   
                                                 isEntityD                                                                                 NaN   
                                                 isEntityP                                                                                 NaN   

                                                                                                               
                                                                                            Plaintiff Victory  
                                                 \emph{Independent Variable}                                   
Panel A: Pre-treatment Outcomes                  twenty_seventeen_group_2_crimes_50m             1.599600e-01  
                                                 pre_treatment_change_in_group_2_crimes_50m      4.083364e-01  
Panel B: Census Tract Characteristics            frac_coll_plus2010                              2.403331e-01  
                                                 job_density_2013                                1.626836e-01  
                                                 med_hhinc2016                                   1.910501e-01  
                                                 poor_share2010                                  1.428679e-01  
                                                 popdensity2010                                  5.946111e-02  
                                                 rent_twobed2015                   

In [4]:
balance_table, pre_treatment_covariates = test_balance(df, analysis, covariates_exploration_df, OUTPUT_TABLES)
balance_table

ValueError: zero-size array to reduction operation maximum which has no identity

In [7]:
# Store treatment date variable -- may be either file_date or latest_docket_date -- and drop rows where missing.
treatment_date_variable = 'file_date'
treatment_month_variable = treatment_date_variable.replace('date', 'month')

# Generate indicators for missing data.
add_missing_indicators(df, ['rent_twobed2015'], pre_treatment_covariates)

# Prepare df for use with the differences package.
df = prepare_df(df, analysis, treatment_date_variable, pre_treatment_covariates, value_vars_crime, month_to_int_dictionary)

NameError: name 'pre_treatment_covariates' is not defined

In [6]:
# Run DiD without conditioning on covariates.
att_gt = ATTgt(data=df, cohort_name=treatment_month_variable, freq='M')
result = att_gt.fit(formula=analysis, control_group='never_treated', n_jobs=-1)

ValueError: DataFrame must be MultiIndex with two levels: entity & time

In [ ]:
# Plot unconditional ATT(t-g)s on a long horizon.
aggregate_by_event_time_and_plot(att_gt, OUTPUT_FIGURES, "att_gt_unconditional_event_study_long_horizon.png",
                                 start_period=-12, end_period=20,
                                 title="Unconditional Estimates of ATT(g, t), Aggregated by Month Relative to Treatment \n (Long Horizon)",
                                 treatment_month_variable=treatment_month_variable, df=df)

In [ ]:
# Plot unconditional ATT(t)s.
aggregate_by_time_and_plot(att_gt, int_to_month_dictionary, OUTPUT_FIGURES, "att_gt_unconditional_time.png",
                           title="Unconditional Estimates of ATT(g, t), Aggregated by Month")

In [ ]:
# Run DiD conditional on covariates.
att_gt = ATTgt(data=df, cohort_name=treatment_month_variable, freq='M')
formula = f'{analysis} ~ ' + '+'.join(pre_treatment_covariates)
result = att_gt.fit(formula=formula, control_group='never_treated', n_jobs=-1)

In [ ]:
# Plot D.R. ATT(t-g)s on a long horizon.
aggregate_by_event_time_and_plot(att_gt, OUTPUT_FIGURES, "att_gt_dr_event_study_long_horizon.png", start_period=-12,
                                 end_period=20,
                                 title="D.R. Estimates of ATT(g, t), Aggregated by Month Relative to Treatment \n (Long Horizon)",
                                 treatment_month_variable=treatment_month_variable, df=df)

In [ ]:
# Plot D.R. ATT(t)s.
aggregate_by_time_and_plot(att_gt, int_to_month_dictionary, OUTPUT_FIGURES, "att_gt_dr_time.png",
                           title="D.R. Estimates of ATT(g, t), Aggregated by Month")